In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


data = np.load('../data/SMP22to95_edge_seq.npy', allow_pickle=True)

ccodes = pd.read_csv('filtros/ccode_reindexed.csv')
pcode_four_digit = pd.read_csv('filtros/pcode_reindexed_four_digit.csv', dtype={'original_code': str})
pcode_descriptions = pd.read_csv('filtros/pcode_reindexed.csv', dtype={'original_code': str})
pcode_descriptions['original_code'] = pcode_descriptions['original_code'].apply(lambda x: x[:4])
años = list(range(1995, 1995 + 28))

In [2]:
def obtain_product_and_country_from_edge(u,v):
    # teniendo en cuenta que v_adj = v - (max_u + 1) y max_u = 136
    original_v =  v + 137
    country = ccodes[ccodes['code'] == u]['country_name'].iloc[0]
    product_code = pcode_four_digit[pcode_four_digit['code'] == original_v]['original_code'].iloc[0]
    product_descriptions = pcode_descriptions[pcode_descriptions['original_code'] == product_code]['description']


    print(f"Pais: {country} \n")

    print(product_code)
    print(product_descriptions)

In [3]:
#matrix values
max_u = 136
max_v = 1354
num_v = max_v - max_u  #real quantity of v nodes

adj_matrices = []

for year_data in data:

    adj_matrix = np.zeros((max_u + 1, num_v))

    for u, v, w in year_data:
        v_adj = v - (max_u + 1)  #adjust v index
        if w >= 1:
            weight = 1
        else:
            weight = 0
        adj_matrix[u, v_adj] = weight  #fill with binarized rca

    adj_matrices.append(adj_matrix)

In [ ]:
#matrix values
max_u = 136
max_v = 1354
num_v = max_v - max_u  #real quantity of v nodes

adj_matrices_nb = []

for year_data in data:

    adj_matrix_nb = np.zeros((max_u + 1, num_v))

    for u, v, w in year_data:
        v_adj = v - (max_u + 1)  #adjust v index
        adj_matrix_nb[u, v_adj] = w  #fill with binarized rca

    adj_matrices_nb.append(adj_matrix_nb)


In [60]:
adj_Test = (adj_matrices_nb[22] >= 1).astype(int)


In [67]:
adj_Test = (adj_matrices_nb[22] >= 1).astype(int)
print(np.array_equal(adj_Test, adj_matrices[22]))


True


In [12]:
def probability_rca(product,year):
    adj_matrix = adj_matrices[year]
    frecuency = 0
    for country in range(137):
        frecuency += adj_matrix[country,product]
    return frecuency/137

In [13]:
probabilities_array_22 = np.zeros((1218))
for i in range(1218):
        probabilities_array_22[i] = probability_rca(i,22) #es bastante mas eficiente guardarlo en un array ya que reutilizamos los resultados

In [ ]:
def probability_intersection_rca(product_i, product_j, year):
    adj_matrix = adj_matrices[year]
    frecuency = 0
    for country in range(137):
        frecuency += adj_matrix[country,product_i] * adj_matrix[country,product_j]
    return frecuency/137

def similarity(product_i, product_j, year, probability_array):
    intersection = probability_intersection_rca(product_i,product_j,year) 
    prob_ij = intersection/ probability_array[product_i] #intersection/probability_rca(product_i,year)
    prob_ji = intersection/probability_array[product_j] #intersection/probability_rca(product_j,year)
    return min(prob_ij,prob_ji)

In [15]:
def similarity_matrix_for(year,probabilities_array_22):
    similarity_matrix = np.zeros((1218,1218))
    for i in range(1218):
        for j in range(1218):
            if i != j:
                similarity_matrix[i,j] = similarity(i,j,year,probabilities_array_22)
            else:
                similarity_matrix[i,j] = 1
    return similarity_matrix

In [16]:
similarity_22 = similarity_matrix_for(22,probabilities_array_22)
print(similarity_22)

[[1.         0.175      0.28571429 ... 0.33333333 0.17857143 0.18181818]
 [0.175      1.         0.25       ... 0.175      0.225      0.125     ]
 [0.28571429 0.25       1.         ... 0.22222222 0.03571429 0.09090909]
 ...
 [0.33333333 0.175      0.22222222 ... 1.         0.32142857 0.36363636]
 [0.17857143 0.225      0.03571429 ... 0.32142857 1.         0.5       ]
 [0.18181818 0.125      0.09090909 ... 0.36363636 0.5        1.        ]]


In [25]:
print(similarity_22[1])

[0.175 1.    0.25  ... 0.175 0.225 0.125]


In [76]:
print(sorted(similarity_22[1009], reverse = True))

[np.float64(1.0), np.float64(0.7000000000000001), np.float64(0.6363636363636364), np.float64(0.6000000000000001), np.float64(0.6000000000000001), np.float64(0.5833333333333333), np.float64(0.5454545454545455), np.float64(0.5454545454545455), np.float64(0.5384615384615384), np.float64(0.5384615384615384), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.5), np.float64(0.4666666666666666), np.float64(0.4666666666666666), np.float64(0.46153846153846156), np.float64(0.46153846153846156), np.float64(0.46153846153846156), np.float64(0.45454545454545453), np.float64(0.45454545454545453), np.float64(0.4444444444444444), np.float64(0.4375), np.float64(0.4285714285714286), np.float64(0.4285714285714286), np.float64(0.42105263157894735), np.float64(0.41666666666666663), np.float64(0.41666666666666663), np.float64(0.41666666666666663), np.float64(0.41666666666666663), np.float64(0.41666666666666663), np.float64(0.41

In [18]:
def density_of(country, product, similarity_matrix, adj_matrix):
    filtrado = 0
    total = 0
    for i in range(1218):
         similarity = similarity_matrix[i,product] 
         total += similarity
         filtrado += similarity*adj_matrix[country,i]
    return filtrado/total

In [81]:
print(density_of(3,0,similarity_22, adj_matrices[22]))

0.03072855586542011


In [19]:
density_matrix = np.zeros((max_u + 1, num_v))

for i in range(137):
    for j in range(1218):
        density_matrix[i,j] = density_of(i,j,similarity_22,adj_matrices[22])

print(density_matrix)

[[0.05447149 0.08010418 0.04261529 ... 0.05280971 0.10408435 0.08924364]
 [0.0104126  0.02044892 0.01264375 ... 0.01101569 0.01770173 0.01545028]
 [0.00734382 0.01106973 0.00660599 ... 0.00823717 0.01084228 0.00984441]
 ...
 [0.0211338  0.03214019 0.02327985 ... 0.01747742 0.02975344 0.02764586]
 [0.04364517 0.07599036 0.03504518 ... 0.04225052 0.07643659 0.06667652]
 [0.0463757  0.07981211 0.04253336 ... 0.04258889 0.07188541 0.05471706]]


In [24]:
print(sorted(density_matrix[130], reverse = True))

[np.float64(0.5886950085669314), np.float64(0.5881720432804001), np.float64(0.572284861888211), np.float64(0.570996207900359), np.float64(0.5644237924458538), np.float64(0.5602270355407701), np.float64(0.559731392889622), np.float64(0.5557095654933907), np.float64(0.5527579668545576), np.float64(0.5472586822163923), np.float64(0.5459829603813431), np.float64(0.5450187760481711), np.float64(0.5436311824759166), np.float64(0.5405138927088854), np.float64(0.5383650466208423), np.float64(0.5377744495327499), np.float64(0.5373707396912102), np.float64(0.5371329670243823), np.float64(0.5370316466212305), np.float64(0.5348201102734911), np.float64(0.533847487601473), np.float64(0.5326033035201495), np.float64(0.5317830846892171), np.float64(0.530846648305681), np.float64(0.5276617060215747), np.float64(0.526585243632995), np.float64(0.5262815791442288), np.float64(0.5260209510752304), np.float64(0.5241024097199716), np.float64(0.5236666258222629), np.float64(0.5224462809878608), np.float64(0.

In [135]:
obtain_product_and_country_from_edge(129,1)

Pais: United Rep. of Tanzania 

0102
3     Bovine animals: live, pure-bred breeding animals
4    Bovine animals: live, other than pure-bred bre...
Name: description, dtype: object


In [ ]:
X = density_matrix.flatten().reshape(-1, 1) # input: densidad
y = adj_matrices[23].flatten() # output: rca binario (0 o 1)


#train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#training
model = LogisticRegression()
model.fit(X_train, y_train)

#Predicción
y_pred = model.predict(X_test)

#Metricas
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.89      0.97      0.93     28742
         1.0       0.61      0.27      0.38      4632

    accuracy                           0.87     33374
   macro avg       0.75      0.62      0.65     33374
weighted avg       0.85      0.87      0.85     33374

Accuracy: 0.8745430574698867


In [66]:
#train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#training
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

#Predicción
y_pred = model.predict(X_test)

#Metricas
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.94      0.79      0.86     28742
         1.0       0.35      0.69      0.46      4632

    accuracy                           0.78     33374
   macro avg       0.64      0.74      0.66     33374
weighted avg       0.86      0.78      0.80     33374

Accuracy: 0.7753340924072631


In [ ]:
X = adj_matrices_nb[22].flatten().reshape(-1, 1) # input: densidad
y = adj_matrices[23].flatten() # output: rca binario (0 o 1)


#train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#training
model = LogisticRegression()
model.fit(X_train, y_train)

#Predicción
y_pred = model.predict(X_test)

#Metricas
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97     28742
         1.0       0.91      0.66      0.77      4632

    accuracy                           0.94     33374
   macro avg       0.93      0.82      0.87     33374
weighted avg       0.94      0.94      0.94     33374

Accuracy: 0.9441181758254928


In [ ]:
#Densidad como feature
densities = density_matrix.flatten().reshape(-1, 1)

#RCA continuo como feature
rca_values = adj_matrices_nb[22].flatten().reshape(-1, 1) 

# rca binario
y = adj_matrices[23].flatten()  
#features: densidad y RCA
X = np.hstack([densities, rca_values])  

#Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train
model = LogisticRegression()
model.fit(X_train, y_train)

#Predicción
y_pred = model.predict(X_test)

#Métricas de clasficiación
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))



              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97     28742
         1.0       0.92      0.71      0.80      4632

    accuracy                           0.95     33374
   macro avg       0.94      0.85      0.89     33374
weighted avg       0.95      0.95      0.95     33374

Accuracy: 0.9508899143045485


In [72]:
#Densidad como feature
densities = density_matrix.flatten().reshape(-1, 1)

#RCA continuo como feature
rca_values = adj_matrices_nb[22].flatten().reshape(-1, 1) 

# rca binario
y = adj_matrices[23].flatten()  
#features: densidad y RCA
X = np.hstack([densities, rca_values])  

#Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

#Predicción
y_pred = model.predict(X_test)

#Métricas de clasficiación
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))



              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97     28742
         1.0       0.75      0.91      0.82      4632

    accuracy                           0.95     33374
   macro avg       0.87      0.93      0.90     33374
weighted avg       0.95      0.95      0.95     33374

Accuracy: 0.9459759093905435
